# Matching Algorithm Model Comparison

Here you will find a comprehensive comparison using our existing match data and testing on various models along with annotation describing each model. The next chunk of code here is just importing various 

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from numpy.random import randn
from numpy.matlib import repmat

from scipy.stats import norm
from scipy.optimize import fmin
from scipy.special import erf

from patsy import dmatrices

from sklearn import metrics
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
a = pd.DataFrame([[np.nan, 2, 3], [4, np.nan, 6]])

a[0] = a[0].fillna(0)

In [3]:
def array_vector(col):
    return np.array(str(col))

arrayerize = np.vectorize(array_vector)

def one_hot_encode(df, column, labels_column=None, whitelist=[]):
    # This is gross but since strings are iterable, we have to wrap them in a list
    # in order for the binarizer to parse the labels as strings and not chars
    labels = arrayerize(pd.DataFrame(df[column]))
    terms = arrayerize(pd.DataFrame(list(set(df[column]))))

    mlb = MultiLabelBinarizer()
    mlb.fit(terms)
    mlb.transform(labels)
    columns = [ f'{column}-{classname}' for classname in  mlb.classes_]

    encoded = pd.DataFrame(mlb.transform(labels), columns=columns, index=df[column].index)

    df.drop(column, axis=1, inplace=True)
    return df.join(encoded)

In [4]:
firms = pd.read_csv('../match-data/match-v1/firms.csv')
jobs = pd.read_csv('../match-data/match-v1/job-openings.csv')
jobs_parent = pd.read_csv('../match-data/match-v1/job-openings-parent.csv')
job_seekers = pd.read_csv('../match-data/match-v1/job-seekers.csv')
matches = pd.read_csv('../match-data/match-v1/matches.csv')
matches_parent = pd.read_csv('../match-data/match-v1/matches-parent.csv')

ignore = ['number', 'caseid', 'parent_caseid', 'job_id', 'hired_yes_no', 'quit', 'fired']

job_seekers.columns = ['JS-' + c if c not in ignore else c for c in job_seekers.columns]
firms.columns = ['JOB-' + c if c not in ignore else c for c in firms.columns]
jobs.columns = ['JOB-' + c if c not in ignore else c for c in jobs.columns]
matches.columns = ['MATCH-' + c if c not in ignore else c for c in matches.columns]

matches_merged = pd.merge(matches, matches_parent, on='number')
jobs_merged = pd.merge(jobs, jobs_parent, on='number')

In [5]:
categorical_columns = ["JS-gender", "JS-highest_edu_level", "JS-nationality", "JS-gendermix_not_allowed", "JS-benefit1", "JS-benefit2", "JS-city", "JS-english_proficiency", "JS-impairments", "JS-major", "JS-opposite_gender_coworkers", "JS-opposite_gender_manager", "JS-first_job_field_preference", "JS-second_job_field_preference", "JOB-bus_covered", "JOB-childcare_subsidy_offered", "JOB-dorm_covered", "JOB-driving_ability_required", "JOB-education_required", "JOB-english_proficiency_required", "JOB-female_requied", "JOB-free_meals_at_wok", "JOB-health_insurance_offered", "JOB-hearing_disability_accepted", "JOB-housing_subsidy_offered", "JOB-it_proficiency_required", "JOB-job_category", "JOB-job_description", "JOB-job_production", "JOB-jordanian_experience_required", "JOB-literacy_required", "JOB-male_required", "JOB-meal_subsidy_offered", "JOB-night_shifts_required", "JOB-noncognitive_skill_preference1", "JOB-noncognitive_skill_preference2", "JOB-numeracy_requied", "JOB-physical_disability_accepted", "JOB-physical_work_abilities_required", "JOB-problem_solving_required", "JOB-school_subsidy_offered", "JOB-specialization_required", "JOB-speech_disability_accepted", "JOB-syrian_considered", "JOB-transport_subsidy_offered", "JOB-visual_disability_accepted", "JOB-work_permit_offered",]
all_columns = ["JS-age", "JS-gender", "JS-highest_edu_level", "JS-will_work_night_shift", "JS-nationality", "JS-gendermix_not_allowed", "JS-will_work_qiz", "JS-arab_coworkers", "JS-benefit1", "JS-benefit2", "JS-city", "JS-daily_hours_willing_to_work", "JS-days_willing_train_unpaid", "JS-distance_willing_to_travel", "JS-english_proficiency", "JS-experience_clerical_work", "JS-experience_factory", "JS-experience_management_work", "JS-experience_manual_labor", "JS-experience_professional_work", "JS-follow_up_agreement", "JS-has_job", "JS-impairments", "JS-major", "JS-nonarab_coworkers", "JS-opposite_gender_coworkers", "JS-opposite_gender_manager", "JS-weekly_days_willing_to_work", "JS-will_live_in_dorm", "JS-will_train_unpaid", "JS-years_education", "JS-years_exp", "JS-first_job_field_preference", "JS-rwage1", "JS-second_job_field_preference", "MATCH-hh_income", "MATCH-interest_applying", "MATCH-num_children", "MATCH-personal_income", "JOB-bus_covered", "JOB-childcare_subsidy_offered", "JOB-dorm_covered", "JOB-driving_ability_required", "JOB-education_required", "JOB-english_proficiency_required", "JOB-female_requied", "JOB-free_meals_at_wok", "JOB-health_insurance_offered", "JOB-hearing_disability_accepted", "JOB-housing_subsidy_offered", "JOB-it_proficiency_required", "JOB-job_category", "JOB-job_description", "JOB-job_production", "JOB-jordanian_experience_required", "JOB-literacy_required", "JOB-male_required", "JOB-meal_subsidy_offered", "JOB-night_shifts_required", "JOB-noncognitive_skill_preference1", "JOB-noncognitive_skill_preference2", "JOB-num_vacancies", "JOB-numeracy_requied", "JOB-physical_disability_accepted", "JOB-physical_work_abilities_required", "JOB-problem_solving_required", "JOB-school_subsidy_offered", "JOB-specialization_required", "JOB-speech_disability_accepted", "JOB-syrian_considered", "JOB-transport_subsidy_offered", "JOB-visual_disability_accepted", "JOB-wage_offered", "JOB-work_permit_offered", "JOB-years_experience_required", 'hired_yes_no', 'quit', 'fired']
scalar_columns = ["JS-age", "JS-daily_hours_willing_to_work", "JS-days_willing_train_unpaid", "JS-distance_willing_to_travel", "JS-years_education", "JS-years_exp", "JS-rwage1", "MATCH-hh_income", "JS-num_children", "JS-personal_income", "JOB-num_vacancies", "JOB-wage_offered"]

In [6]:
job_seekers['parent_case_id'] = job_seekers['caseid']

In [7]:
merged = pd.merge(job_seekers, matches_merged, on='parent_case_id')
merged = pd.merge(merged, jobs_merged, on='job_id')

merged.to_csv('../match-data/match-v1/merged.csv')

In [8]:
formatted = pd.DataFrame()
for col in all_columns:
    formatted[col] = merged[col]

dvs = ['hired_yes_no', 'quit', 'fired']

for col in all_columns:
    if col not in scalar_columns and col not in dvs:
        formatted = one_hot_encode(formatted, col)

for col in formatted.columns:
    # TODO: Change this
    formatted[col] = formatted[col].replace(['---'], 0)
    
formatted.to_csv('../match-data/match-v1/formatted.csv')

In [9]:
formatted['hired_yes_no'] = formatted['hired_yes_no'].fillna(0)
formatted['hired_yes_no'] = formatted['hired_yes_no'].replace(['---'], 0)

formatted['quit'] = formatted['quit'].fillna(0)
formatted['quit'] = formatted['quit'].replace(['---'], 0)
formatted['quit'] = formatted['quit'].replace(['no'], 0)
formatted['quit'] = formatted['quit'].replace(['yes'], 1)

formatted['fired'] = formatted['fired'].fillna(0)
formatted['fired'] = formatted['fired'].replace(['---'], 0)
formatted['fired'] = formatted['fired'].replace(['no'], 0)
formatted['fired'] = formatted['fired'].replace(['yes'], 1)

formatted['hired_yes_no'] = formatted['hired_yes_no'].astype(bool)
formatted['quit'] = formatted['quit'].astype(bool)
formatted['fired'] = formatted['fired'].astype(bool)

outcomes = pd.DataFrame()
outcomes['retained'] = formatted['hired_yes_no'] & ~(formatted['quit'] | formatted['fired'])
outcomes['hired'] = formatted['hired_yes_no']
y = pd.DataFrame()
o = []
for index, row in outcomes.iterrows():
    if row['hired'] and row['retained']:
        o.append(2)
    elif not row['hired'] and not row['retained']:
        o.append(0)
    elif row['hired'] and not row['retained']:
        o.append(1)

y['outcomes'] = o

In [10]:
formatted = formatted.drop(dvs, axis=1)

In [11]:
to_drop = []
for col in formatted.columns:
    if col in scalar_columns:
        formatted[col] = formatted[col].astype(float)
        mean = formatted[col].mean()
        formatted[col] = formatted[col].replace(['---'], mean)
        formatted[col] = formatted[col].fillna(mean)
    else:
        formatted[col] = formatted[col].fillna(0)
        formatted[col] = formatted[col].astype(int)
        formatted[col] = formatted[col].replace(['---'], 0)
    
    if col.endswith('---') or col.endswith('nan'):
        to_drop.append(col)

formatted = formatted.drop(columns=to_drop)

In [12]:
y=y.astype('int')
y.to_csv('../match-data/match-v1/y.csv')

In [13]:
formatted.to_csv('../match-data/match-v1/X.csv')
X = formatted

In [14]:
print(X.shape, y.shape)

(112, 225) (112, 1)


## Random Forest

Here we train a random forest model with a maximum tree depth of 10 using 100 decision trees in our forest on our training set. We train and test this 100 times using random splits between our train and test. The average accuracy of those 100 tests is logged in the output below. As you can see it's around 75% accuracy.

In [15]:
from sklearn.ensemble import RandomForestClassifier
scores = []
model = RandomForestClassifier(max_depth=10, random_state=0, n_estimators=100)

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    model.fit(X_train, y_train)
    s = model.score(X_test, y_test)
    scores.append(s)
    
sum(scores) / len(scores)

0.7556756756756756

To get a better idea of what our end result decision tree looks like, the below image (if it's too small to read, right click the image and click "view image") gives us a visual representation of our decision tree. Each property being evaluated is either prefixed with "JS-" (job seeker), "JOB-", or "MATCH-" to denote which data set it came from. For categorical variables, the category is at the end of the property.

For instance `JS-english_proficiency_required-no <= 0.5` means if the job seeker answered `no` to `english_proficiency_required` then choose the node on the right, otherwise, choose the node on the left 

The leaf nodes are essentially what class we think the candidate will belong to with this given match. 

 - Class `0` is rejected
 - Class `1` is hired but not retained
 - Class `2` is both hired and retained
 
Retained in the case of our training set is all candidates that have been retained to date *not* retained for 3 months

In [16]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model.fit(X_train, y_train)
# Extract single tree
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = [str(c) for c in model.classes_],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

The table output of the below code shows the predicted probabilities of our test set (given the above decision tree) along with both the true-DV as well as our prediciton of the DV

In [17]:
probs = model.predict_proba(X_test)
predictions = pd.DataFrame(data=probs, columns=['0', '1', '2'])

predictions['true-DV'] = [v[0] for v in y_test.values]
predictions['predicted-DV'] = model.predict(X_test)
predictions

,0,1,2,true-DV,predicted-DV
0,0.150000,0.050000,0.800000,2,2
1,0.231667,0.432500,0.335833,2,1
2,0.092500,0.220000,0.687500,2,2
3,0.195000,0.065000,0.740000,0,2
4,0.095000,0.330000,0.575000,2,2
5,0.185000,0.040000,0.775000,2,2
6,0.053333,0.145000,0.801667,2,2
7,0.130000,0.030000,0.840000,2,2
8,0.135833,0.260833,0.603333,2,2
9,0.803333,0.050000,0.146667,0,0


In [18]:
importances = {}
for i, importance in enumerate(model.feature_importances_):
    importances[X.columns[i]] = importance
    
importances

c = {}
for k, v in importances.items():
    c[k] = [v]
    

importance_frame = pd.DataFrame.from_dict(c)
importance_frame.to_csv('../match-data/match-v1/random-forest-importance.csv')

# GBRT

Below we have done the same average of 100 tests using a GBRT model

In [19]:
scores = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

sum(scores) / len(scores)

0.7067567567567563

## LASSO CV

Below we have done the same average of 100 tests using a LASSO model with cross validation. I have no idea why this model is performing so bad

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
scores = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    model = LassoCV(cv=3, normalize=True).fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

sum(scores) / len(scores)

0.29241915062796164

# K Best Feature Elimination

Below we are using K-Best (k = 10) as a means of selecting features for a logit model. The output shows the columns which were selected

In [21]:
from sklearn.feature_selection import SelectKBest

# Create and fit selector
selector = SelectKBest(chi2, k=10)
selector.fit(X, y)
# Get idxs of columns to keep
idxs_selected = selector.get_support(indices=True)
cols = []
for i, c in enumerate(X.columns):
    if i in idxs_selected:
        cols.append(c)

cols

['JS-age',
 'JS-days_willing_train_unpaid',
 'JS-distance_willing_to_travel',
 'JS-years_exp',
 'MATCH-hh_income',
 'JOB-english_proficiency_required-not_literate',
 'JOB-it_proficiency_required-no',
 'JOB-it_proficiency_required-none',
 'JOB-jordanian_experience_required-no',
 'JOB-problem_solving_required-no']

### Accuracy using top 10 k-best features


Below we have done the same average of 100 tests using a logit model with cross validation using only the 10 features selected using K-Best.

In [22]:
model = LogisticRegressionCV(max_iter=1000)

scores = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    XX_train = pd.DataFrame()
    XX_test = pd.DataFrame()
    for c in cols:
        XX_train[c] = X_train[c]
        XX_test[c] = X_test[c]

    model.fit(XX_train, y_train)
    scores.append(model.scores(XX_test, y_test))

sum(scores) / len(scores)

AttributeError: 'LogisticRegressionCV' object has no attribute 'scores'

### Export Coefs & p-values to csv

this is just exporting the coefs, p-values, and odds ratio to csv

In [ ]:
coef_dict = {}
for coef, feat in zip(model.coef_[0], cols):
    coef_dict[feat] = coef
    
c = {}
for k, v in coef_dict.items():
    c[k] = [v]
    

coef_frame = pd.DataFrame.from_dict(c)
sorted_frame = coef_frame.columns[coef_frame.ix[coef_frame.last_valid_index()].argsort()]
coef_frame.to_csv('../match-data/match-v1/coefs.csv')

odds_ratios = np.exp(coef_frame)
scores, pvalues = chi2(X, y)

p_dict = {}
for pvalue, feat in zip(pvalues, cols):
    p_dict[feat] = pvalue
    
c = {}
for k, v in p_dict.items():
    c[k] = [v]
    

pval_frame = pd.DataFrame.from_dict(c)
sorted_frame = pval_frame.columns[pval_frame.ix[pval_frame.last_valid_index()].argsort()]
pval_frame.to_csv('../match-data/match-v1/pvalues.csv')

In [ ]:
display(coef_frame)
display(pval_frame)
display(odds_ratios)

In [ ]:
combined = coef_frame.append(pval_frame).append(np.exp(odds_ratios))
combined.to_csv('../match-data/match-v1/combined.csv')

# Recursive Feature Elimination

Here we are using recursive feature elimination in order to select our features. As you can see the features are quite different from K-Best

In [ ]:
model = LogisticRegressionCV(max_iter=1000)
from sklearn.feature_selection import RFE
selector = RFE(model, 10, step=1)
selector.fit(X, y)

In [ ]:
len(selector.support_)
cols = []
for i, v in enumerate(selector.support_):
    if v:
        cols.append(X.columns[i])

cols

### Accuracy using top 10 RFE features

Below we are fitting a logit with these features and it's performing much worse than K-Best

In [ ]:
model = LogisticRegressionCV(max_iter=1000)

scores = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    XX_train = pd.DataFrame()
    XX_test = pd.DataFrame()
    for c in cols:
        XX_train[c] = X_train[c]
        XX_test[c] = X_test[c]

    model.fit(XX_train, y_train)
    scores.append(model.scores(XX_test, y_test))

sum(scores) / len(scores)